In [4]:
import numpy as np
import galois
import sympy as sp

In [2]:
p = 3 
e = 2
d = p**e
F = galois.GF(d, irreducible_poly='x^2+x+2')
# F = galois.GF(d)
F.elements

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


GF([0, 1, 2, 3, 4, 5, 6, 7, 8], order=3^2)

First step is to check if the displacement operators are the same as the translation operators. First we have to define the generalized Pauli matrices. Wootters doesn't actually say how, but they are constructed as follows:

In [18]:
II = sp.eye(p)

def tensorproduct(a, b):
    l = len(a)
    return sp.ImmutableMatrix((sp.tensorproduct(a, b)).reshape(l, l))

def lift(a):
    return np.int64(np.array(a))

def omega(a):
    return sp.exp(sp.I * 2 * sp.pi * lift(a) / p)

In [22]:
FF = sp.zeros(p, p)
for m in range(p):
    for n in range(3):
        FF += omega(F.elements[m] * F.elements[n]) * tensorproduct(II[:, m], sp.adjoint(II[:, n]))
FF = FF / sp.sqrt(p)

In [24]:
Z = sp.zeros(p, p)
X = sp.zeros(p, p)
for n in range(p):
    v = II[:, n]
    fv = FF * v
    Z += omega(F.elements[n]) * tensorproduct(v, sp.adjoint(v))
    X += omega(-F.elements[n]) * tensorproduct(fv, sp.adjoint(fv))
X = X.applyfunc(sp.nsimplify)

In [25]:
X

Matrix([
[0, 0, 1],
[1, 0, 0],
[0, 1, 0]])

In [26]:
Z

Matrix([
[1,             0,              0],
[0, exp(2*I*pi/3),              0],
[0,             0, exp(-2*I*pi/3)]])

Now Wootters creates the displacement operators using tensor products of the generalized Pauli matrices. First we must obtain a basis decomposition of the phase point $(\alpha,\beta)$.

In [148]:
basis = F([F.primitive_element**i for i in range(e)])
basis

GF([1, 3], order=3^2)

In [149]:
g = F.Zeros((e,e))
for i in range(e):
    for j in range(e):
        g[i,j] = (F.primitive_element**(i+j)).field_trace()
g

GF([[2, 2],
    [2, 0]], order=3^2)

In [150]:
G = np.linalg.inv(g)
G

GF([[0, 2],
    [2, 1]], order=3^2)

In [151]:
dual_basis = G @ basis
dual_basis

GF([6, 5], order=3^2)

In [152]:
basis = basis[::-1]
dual_basis = dual_basis[::-1]
print(basis, dual_basis)

[3 1] [5 6]


In [157]:
# sanity check
for i, a in enumerate(basis):
    for j, b in enumerate(dual_basis):
        if not (a*b).field_trace() == (i == j):
            raise Exception('Not dual basis!')

In [196]:
F.elements[8] * dual_basis

GF([3, 1], order=3^2)

Los operadores de traslación están dados: $$T(x,y) = X^{x_1} Z^{y_1} \otimes \cdots \otimes X^{x_n} Z^{y_n}$$ donde las potencias de $X$ son los componentes de $x$ respecto a la base y las potencias de $Z$ son los componentes duales de $y$.

In [158]:
def components(x, basis):
    return F([(x * be).field_trace() for be in basis])

In [201]:
[components(x, dual_basis) for x in F([4,4])]

[GF([1, 1], order=3^2), GF([1, 1], order=3^2)]

In [208]:
from sympy.physics.quantum.matrixutils import matrix_tensor_product
from sympy.physics.quantum import TensorProduct

def T(x, y):
    xs = components(x, dual_basis)
    ys = components(y, dual_basis)
    t  = sp.eye(1)
    for i in range(e):
        t = matrix_tensor_product(t, X**lift(xs[i]) * Z**lift(ys[i]))
        # t = TensorProduct(t, X**lift(xs[i]) * Z**lift(ys[i]))
    return t

In [217]:
point = F([F.elements[1], F.elements[2]])
t1 = T(*(3*point))
t2 = T(*(4*point))
((t1 * t2) - (t2 * t1)).applyfunc(sp.nsimplify)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [218]:
T(F.elements[1], F.elements[2])

Matrix([
[0,              0, exp(2*I*pi/3), 0,              0,             0, 0,              0,             0],
[1,              0,             0, 0,              0,             0, 0,              0,             0],
[0, exp(-2*I*pi/3),             0, 0,              0,             0, 0,              0,             0],
[0,              0,             0, 0,              0, exp(2*I*pi/3), 0,              0,             0],
[0,              0,             0, 1,              0,             0, 0,              0,             0],
[0,              0,             0, 0, exp(-2*I*pi/3),             0, 0,              0,             0],
[0,              0,             0, 0,              0,             0, 0,              0, exp(2*I*pi/3)],
[0,              0,             0, 0,              0,             0, 1,              0,             0],
[0,              0,             0, 0,              0,             0, 0, exp(-2*I*pi/3),             0]])